In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium==4.26.1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,618 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [3]:
# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.


In [4]:
# Import modules
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from google.colab import drive
import os
import json

In [5]:
# Install the correct version of chromedriver
chromedriver_autoinstaller.install()

In [6]:
# Set up the Chrome driver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [14]:
driver = webdriver.Chrome(options=options)

In [8]:
# JSON 파일 로드
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/harris_youtube_url_title_date.json', 'r') as f:
    video_data = json.load(f)

In [9]:
# 결과를 저장할 리스트와 파일 경로 설정
output_data = []
output_file_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/comments_data.jsonl'
processed_videos_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/processed_videos.json'

In [10]:
# 이미 처리된 비디오 URL 로드
if os.path.exists(processed_videos_path):
    with open(processed_videos_path, 'r', encoding='utf-8') as f:
        processed_videos = set(json.load(f))
else:
    processed_videos = set()

In [11]:
# 각 비디오에 대해 댓글 스크래핑
for idx, video in enumerate(video_data):
    url = video['url']
    video_title = video['title']
    video_date = video['date']

    if url in processed_videos:
        print(f"Skipping already processed video {idx+1}/{len(video_data)}: {video_title}")
        continue

    try:
        print(f"Processing video {idx+1}/{len(video_data)}: {video_title}")

        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 댓글 로드를 위한 스크롤 다운
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)

        # 모든 댓글을 로드하기 위한 스크롤
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        scroll_attempts = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= 3:
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        # 댓글 요소 찾기
        comments = driver.find_elements(By.CSS_SELECTOR, 'ytd-comment-thread-renderer')
        print(f"Found {len(comments)} comments")

        for comment in comments:
            try:
                # 댓글 텍스트 추출
                comment_text = comment.find_element(By.CSS_SELECTOR, '#content-text').text.strip()

                # 댓글 작성 날짜 추출
                date_element = comment.find_element(By.CSS_SELECTOR, '#published-time-text')
                date = date_element.text.strip()

                # 좋아요 수 추출 (파싱 없이 원본 그대로 저장)
                likes_element = comment.find_element(By.CSS_SELECTOR, '#vote-count-middle')
                likes_text = likes_element.text.strip()

                # 데이터 저장
                output_data.append({
                    'video_url': url,
                    'video_title': video_title,
                    'video_date': video_date,
                    'comment_text': comment_text,
                    'comment_date': date,
                    'comment_likes': likes_text  # 원본 텍스트 그대로 저장
                })
            except Exception as e:
                print(f"Error processing a comment: {e}")
                continue

        # 중간 결과 저장
        with open(output_file_path, 'a', encoding='utf-8') as f_out:
            for data in output_data:
                json_record = json.dumps(data, ensure_ascii=False)
                f_out.write(json_record + '\n')
            output_data = []  # 리스트 초기화

        # 처리된 비디오 URL 저장
        processed_videos.add(url)
        with open(processed_videos_path, 'w', encoding='utf-8') as f_processed:
            json.dump(list(processed_videos), f_processed, ensure_ascii=False)

        print(f"Finished processing video {idx+1}/{len(video_data)}")
    except Exception as e:
        print(f"Error processing video {url}: {e}")
        continue

Processing video 1/126: Good Sense | Harris-Walz 2024
Found 179 comments
Finished processing video 1/126
Processing video 2/126: This Time | Harris-Walz 2024
Found 262 comments
Finished processing video 2/126
Processing video 3/126: Promise | Harris-Walz 2024
Found 101 comments
Finished processing video 3/126
Processing video 4/126: Different | Harris-Walz 2024
Found 184 comments
Finished processing video 4/126
Processing video 5/126: We Rise | Harris-Walz 2024
Found 688 comments
Finished processing video 5/126
Processing video 6/126: Mother | Narrated by Viola Davis for Kamala Harris
Found 336 comments
Finished processing video 6/126
Processing video 7/126: Election Night Preview with Kamala Harris' Campaign Chair Jen O'Malley Dillon
Found 335 comments
Finished processing video 7/126
Processing video 8/126: That Means You | Will Ferrell for Kamala Harris


KeyboardInterrupt: 

In [15]:
# 이미 처리된 비디오 URL 로드
if os.path.exists(processed_videos_path):
    with open(processed_videos_path, 'r', encoding='utf-8') as f:
        processed_videos = set(json.load(f))
else:
    processed_videos = set()

In [16]:
# 각 비디오에 대해 댓글 스크래핑
for idx, video in enumerate(video_data):
    url = video['url']
    video_title = video['title']
    video_date = video['date']

    if url in processed_videos:
        print(f"Skipping already processed video {idx+1}/{len(video_data)}: {video_title}")
        continue

    try:
        print(f"Processing video {idx+1}/{len(video_data)}: {video_title}")

        driver.get(url)
        time.sleep(2)  # 페이지 로드 대기

        # 댓글 로드를 위한 스크롤 다운
        driver.execute_script("window.scrollTo(0, 600);")
        time.sleep(2)

        # 모든 댓글을 로드하기 위한 스크롤
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        scroll_attempts = 0
        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= 3:
                    break
            else:
                last_height = new_height
                scroll_attempts = 0

        # 댓글 요소 찾기
        comments = driver.find_elements(By.CSS_SELECTOR, 'ytd-comment-thread-renderer')
        print(f"Found {len(comments)} comments")

        for comment in comments:
            try:
                # 댓글 텍스트 추출
                comment_text = comment.find_element(By.CSS_SELECTOR, '#content-text').text.strip()

                # 댓글 작성 날짜 추출
                date_element = comment.find_element(By.CSS_SELECTOR, '#published-time-text')
                date = date_element.text.strip()

                # 좋아요 수 추출 (파싱 없이 원본 그대로 저장)
                likes_element = comment.find_element(By.CSS_SELECTOR, '#vote-count-middle')
                likes_text = likes_element.text.strip()

                # 데이터 저장
                output_data.append({
                    'video_url': url,
                    'video_title': video_title,
                    'video_date': video_date,
                    'comment_text': comment_text,
                    'comment_date': date,
                    'comment_likes': likes_text  # 원본 텍스트 그대로 저장
                })
            except Exception as e:
                print(f"Error processing a comment: {e}")
                continue

        # 중간 결과 저장
        with open(output_file_path, 'a', encoding='utf-8') as f_out:
            for data in output_data:
                json_record = json.dumps(data, ensure_ascii=False)
                f_out.write(json_record + '\n')
            output_data = []  # 리스트 초기화

        # 처리된 비디오 URL 저장
        processed_videos.add(url)
        with open(processed_videos_path, 'w', encoding='utf-8') as f_processed:
            json.dump(list(processed_videos), f_processed, ensure_ascii=False)

        print(f"Finished processing video {idx+1}/{len(video_data)}")
    except Exception as e:
        print(f"Error processing video {url}: {e}")
        continue

Skipping already processed video 1/126: Good Sense | Harris-Walz 2024
Skipping already processed video 2/126: This Time | Harris-Walz 2024
Skipping already processed video 3/126: Promise | Harris-Walz 2024
Skipping already processed video 4/126: Different | Harris-Walz 2024
Skipping already processed video 5/126: We Rise | Harris-Walz 2024
Skipping already processed video 6/126: Mother | Narrated by Viola Davis for Kamala Harris
Skipping already processed video 7/126: Election Night Preview with Kamala Harris' Campaign Chair Jen O'Malley Dillon
Processing video 8/126: That Means You | Will Ferrell for Kamala Harris
Found 968 comments
Finished processing video 8/126
Processing video 9/126: Brighter Future | Kamala Harris’ Closing Ad
Found 926 comments
Finished processing video 9/126
Processing video 10/126: A Day in Philadelphia with Kamala Harris | Harris-Walz 2024
Found 241 comments
Finished processing video 10/126
Processing video 11/126: Cardi B On Why She's Supporting Kamala Harris

In [4]:
import pandas as pd

file_path = "/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/harris_youtube_comment_commentdate_likes.jsonl"

df = pd.read_json(file_path, lines=True)

In [5]:
len(df)

52436

In [8]:
print(df.head(5))

                                     video_url                    video_title  \
0  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
1  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
2  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
3  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
4  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   

    video_date                                       comment_text  \
0  Nov 4, 2024                                   HARRIS WALZ 2024   
1  Nov 4, 2024                     Yes who won again?\nTrump gang   
2  Nov 4, 2024  Voting for "kamala Harris" the greatest vote o...   
3  Nov 4, 2024             Kamala , love you \nAlways support you   
4  Nov 4, 2024                              Harris-Walz 2024 🇺🇸🇺🇸   

  comment_date comment_likes  
0  13 days ago            19  
1  12 days ago             8  
2  10

In [7]:
# comment_date 컬럼의 고유한 값 확인
unique_dates = df['comment_date'].unique()

# 고유한 값 출력
print(unique_dates)

['13 days ago' '12 days ago' '10 days ago' '2 weeks ago' '11 days ago'
 '8 days ago' '5 days ago' '2 weeks ago (edited)' '1 month ago'
 '4 days ago' '2 days ago' '23 hours ago' '13 days ago (edited)'
 '3 days ago' '14 hours ago' '4 hours ago' '6 days ago'
 '12 days ago (edited)' '7 days ago' '7 hours ago' '9 days ago'
 '1 day ago' '11 days ago (edited)' '4 days ago (edited)' '19 hours ago'
 '12 hours ago' '1 hour ago' '6 days ago (edited)' '10 hours ago'
 '21 hours ago' '2 hours ago' '18 hours ago' '15 hours ago'
 '10 days ago (edited)' '3 weeks ago' '3 weeks ago (edited)'
 '5 days ago (edited)' '3 days ago (edited)' '11 hours ago' '4 weeks ago'
 '4 weeks ago (edited)' '16 hours ago' '9 hours ago' '8 days ago (edited)'
 '7 days ago (edited)' '5 hours ago' '3 hours ago' '1 month ago (edited)'
 '20 hours ago (edited)' '56 minutes ago' '20 hours ago' '2 months ago'
 '2 months ago (edited)' '3 months ago' '3 months ago (edited)'
 '13 hours ago' '6 hours ago' '22 hours ago']


In [23]:
# comment_date 컬럼의 고유한 값 확인
unique_video_dates = df['video_date'].unique()

# 고유한 값 출력
print(unique_video_dates) #

['Nov 4, 2024' 'Nov 3, 2024' 'Nov 2, 2024' 'Nov 1, 2024' 'Oct 31, 2024'
 'Oct 30, 2024' 'Oct 29, 2024' 'Oct 28, 2024' 'Oct 27, 2024'
 'Oct 26, 2024' 'Oct 25, 2024' 'Oct 24, 2024' 'Oct 23, 2024'
 'Oct 22, 2024' 'Oct 21, 2024' 'Oct 20, 2024' 'Oct 19, 2024'
 'Oct 18, 2024' 'Oct 17, 2024' 'Oct 16, 2024' 'Oct 15, 2024'
 'Oct 14, 2024' 'Oct 13, 2024' 'Oct 12, 2024' 'Oct 11, 2024'
 'Oct 10, 2024' 'Oct 9, 2024' 'Oct 8, 2024' 'Oct 7, 2024' 'Oct 6, 2024'
 'Oct 3, 2024' 'Oct 2, 2024' 'Oct 1, 2024' 'Sep 30, 2024' 'Sep 28, 2024'
 'Sep 27, 2024' 'Sep 24, 2024' 'Sep 23, 2024' 'Sep 20, 2024'
 'Sep 19, 2024' 'Sep 18, 2024' 'Sep 13, 2024' 'Sep 12, 2024'
 'Sep 10, 2024' 'Sep 9, 2024' 'Sep 4, 2024' 'Sep 3, 2024' 'Aug 31, 2024'
 'Aug 28, 2024' 'Aug 23, 2024' 'Aug 15, 2024' 'Aug 11, 2024' 'Aug 6, 2024'
 'Jul 30, 2024' 'Jul 27, 2024' 'Jul 26, 2024' 'Jul 25, 2024']


In [9]:
# 현재 2024년 11월 25일 기준

In [16]:
def parse_comment_date(comment_date):
    try:
        # "(edited)" 제거
        comment_date = re.sub(r"\(edited\)", "", comment_date).strip()

        # "days ago" 형식 처리
        if "day" in comment_date:
            days_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(days=days_ago)

        # "week" 형식 처리
        elif "week" in comment_date:
            weeks_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(weeks=weeks_ago)

        # "month" 형식 처리
        elif "month" in comment_date:
            months_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(days=months_ago * 30)  # 대략 30일 기준

        # "hour" 형식 처리
        elif "hour" in comment_date:
            hours_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(hours=hours_ago)

        # "minute" 형식 처리
        elif "minute" in comment_date:
            minutes_ago = int(re.search(r"\d+", comment_date).group())
            result_date = datetime.now() - timedelta(minutes=minutes_ago)

        # 기본적으로 None 반환 (기타 형식에 대해 처리할 수 없을 경우)
        else:
            return None

        # 날짜만 반환 (시간과 마이크로초 제거)
        return result_date.date()

    except Exception as e:
        print(f"Error parsing comment_date '{comment_date}': {e}")
        return None


In [17]:
# 'comment_date'를 datetime으로 변환하여 새로운 컬럼 생성
df['parsed_comment_date'] = df['comment_date'].apply(parse_comment_date)

In [18]:
# 결과 출력 (테스트용)
print(df[['comment_date', 'parsed_comment_date']].head(10))

  comment_date parsed_comment_date
0  13 days ago          2024-11-11
1  12 days ago          2024-11-12
2  10 days ago          2024-11-14
3  13 days ago          2024-11-11
4  13 days ago          2024-11-11
5  2 weeks ago          2024-11-10
6  12 days ago          2024-11-12
7  13 days ago          2024-11-11
8  2 weeks ago          2024-11-10
9  11 days ago          2024-11-13


In [19]:
# comment_date 컬럼의 고유한 값 확인
unique_dates = df['parsed_comment_date'].unique()

# 고유한 값 출력
print(unique_dates)

# 마지막 비디오 영상이 Jul 25, 2024이며, 크롤링한 2024년 11월 24일 기준 3 month ago 보다 이전의 댓글 날짜는 없다.
# 즉 7월 25일 ~ 8월 26일 -> 8월 26일로 통일된다. (마찬가지로 2024, 8, 27 ~ 2024, 9, 25는 2024, 9, 25로 통일됨.)
# 어차피 댓글 날짜를 추출한 이유는 필터링이 목적이므로 크게 상관없다.

[datetime.date(2024, 11, 11) datetime.date(2024, 11, 12)
 datetime.date(2024, 11, 14) datetime.date(2024, 11, 10)
 datetime.date(2024, 11, 13) datetime.date(2024, 11, 16)
 datetime.date(2024, 11, 19) datetime.date(2024, 10, 25)
 datetime.date(2024, 11, 20) datetime.date(2024, 11, 22)
 datetime.date(2024, 11, 23) datetime.date(2024, 11, 21)
 datetime.date(2024, 11, 24) datetime.date(2024, 11, 18)
 datetime.date(2024, 11, 17) datetime.date(2024, 11, 15)
 datetime.date(2024, 11, 3) datetime.date(2024, 10, 27)
 datetime.date(2024, 9, 25) datetime.date(2024, 8, 26)]


In [25]:
print(df.head(5))

                                     video_url                    video_title  \
0  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
1  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
2  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
3  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
4  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   

    video_date                                       comment_text  \
0  Nov 4, 2024                                   HARRIS WALZ 2024   
1  Nov 4, 2024                     Yes who won again?\nTrump gang   
2  Nov 4, 2024  Voting for "kamala Harris" the greatest vote o...   
3  Nov 4, 2024             Kamala , love you \nAlways support you   
4  Nov 4, 2024                              Harris-Walz 2024 🇺🇸🇺🇸   

  comment_date comment_likes parsed_comment_date  
0  13 days ago            19          2024-11-1

In [26]:
# 'comment_date' 컬럼 삭제
df = df.drop(columns=['comment_date'])

In [27]:
print(df.head(5))

                                     video_url                    video_title  \
0  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
1  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
2  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
3  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   
4  https://www.youtube.com/watch?v=_v_4IgeVQ8M  Good Sense | Harris-Walz 2024   

    video_date                                       comment_text  \
0  Nov 4, 2024                                   HARRIS WALZ 2024   
1  Nov 4, 2024                     Yes who won again?\nTrump gang   
2  Nov 4, 2024  Voting for "kamala Harris" the greatest vote o...   
3  Nov 4, 2024             Kamala , love you \nAlways support you   
4  Nov 4, 2024                              Harris-Walz 2024 🇺🇸🇺🇸   

  comment_likes parsed_comment_date  
0            19          2024-11-11  
1             8       

In [28]:
len(df)

52436

In [29]:
# 날짜 기준 설정 (2024-11-04 이후는 삭제)
date_threshold = datetime(2024, 11, 4).date()

In [30]:
# 'parsed_comment_date'를 datetime 형식으로 변환하여 날짜만 비교
df['parsed_comment_date'] = pd.to_datetime(df['parsed_comment_date']).dt.date

In [31]:
# 날짜 조건에 맞는 행만 남기기 (2024-11-04 이후의 데이터 삭제)
df = df[df['parsed_comment_date'] <= date_threshold]

In [32]:
# comment_date 컬럼의 고유한 값 확인
unique_dates = df['parsed_comment_date'].unique()

# 고유한 값 출력
print(unique_dates)

[datetime.date(2024, 10, 25) datetime.date(2024, 11, 3)
 datetime.date(2024, 10, 27) datetime.date(2024, 9, 25)
 datetime.date(2024, 8, 26)]


In [33]:
len(df)

43894

In [34]:
print(df.head(5))

                                        video_url  \
149   https://www.youtube.com/watch?v=_v_4IgeVQ8M   
3980  https://www.youtube.com/watch?v=81HpMyoXd0g   
3981  https://www.youtube.com/watch?v=81HpMyoXd0g   
3982  https://www.youtube.com/watch?v=81HpMyoXd0g   
3987  https://www.youtube.com/watch?v=81HpMyoXd0g   

                                            video_title   video_date  \
149                       Good Sense | Harris-Walz 2024  Nov 4, 2024   
3980  A Day in Philadelphia with Kamala Harris | Har...  Nov 2, 2024   
3981  A Day in Philadelphia with Kamala Harris | Har...  Nov 2, 2024   
3982  A Day in Philadelphia with Kamala Harris | Har...  Nov 2, 2024   
3987  A Day in Philadelphia with Kamala Harris | Har...  Nov 2, 2024   

                                           comment_text comment_likes  \
149                             In the limelight again?             2   
3980  Vote and bring friends, family or neighbors! F...            20   
3981                         

In [35]:
output_file_path = "/content/drive/MyDrive/[인공지능기초]/sns/youtube/comment/harris_youtube_comment_filtered.jsonl"
df.to_json(output_file_path, orient='records', lines=True)